In [23]:
#import dependencies
%matplotlib notebook
import csv
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt
#from config import api_key
from citipy import citipy
import random
import json
from pprint import pprint
import time
import gmaps

In [45]:
mort_2010 = pd.read_csv('asthma_mortality_2010.csv')
mort_2011 = pd.read_csv('asthma_mortality_2011.csv')
mort_2012 = pd.read_csv('asthma_mortality_2012.csv')
mort_2013 = pd.read_csv('asthma_mortality_2013.csv')
mort_2014 = pd.read_csv('asthma_mortality_2014.csv')

In [46]:
aqi_yearly = pd.read_csv('aqi_state_yearly.csv')

In [48]:
aqi_2010_df = aqi_yearly[['Unnamed: 0','2010']]
aqi_2010_df = aqi_2010_df.rename(columns={'Unnamed: 0': 'state'})
aqi_2011_df = aqi_yearly[['Unnamed: 0','2011']]
aqi_2011_df = aqi_2011_df.rename(columns={'Unnamed: 0': 'state'})
aqi_2012_df = aqi_yearly[['Unnamed: 0','2012']]
aqi_2012_df = aqi_2012_df.rename(columns={'Unnamed: 0': 'state'})
aqi_2013_df = aqi_yearly[['Unnamed: 0','2013']]
aqi_2013_df = aqi_2013_df.rename(columns={'Unnamed: 0': 'state'})
aqi_2014_df = aqi_yearly[['Unnamed: 0','2014']]
aqi_2014_df = aqi_2014_df.rename(columns={'Unnamed: 0': 'state'})